In [ ]:
import pickle
from collections import defaultdict 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from utils.mmd import maximum_mean_discrepancy
import two_moons_lueckmann_numpy, two_moons_wiqvist_numpy
import config

np.random.seed(0)

In [ ]:
plt.rcParams.update({
    "axes.labelsize" : 20,
    "xtick.labelsize" : 16,
    "ytick.labelsize" : 16,
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["times"],
    'text.latex.preamble' : r'\usepackage{{amsmath}}'
})

# Load samples from the approximators

In [ ]:
def load_samples_np_txt(method_abbreviation, dir_prefix="", seeds=None):
    d = defaultdict(dict)
    d['post_samples'] = defaultdict(dict)
    d['lik_samples'] = defaultdict(dict)
    
    if seeds is None:
        seeds = config.seeds

    for seed in seeds:
        d['post_samples']['2000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_round2_seed{seed}.csv", 
                                                     delimiter=",")
        d['post_samples']['6000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_round6_seed{seed}.csv", 
                                                     delimiter=",")
        d['post_samples']['10000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_round10_seed{seed}.csv", 
                                                     delimiter=",")
        try:
            d['lik_samples']['2000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_pred_round2_seed{seed}.csv", 
                                                         delimiter=",")
            d['lik_samples']['6000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_pred_round6_seed{seed}.csv", 
                                                         delimiter=",")
            d['lik_samples']['10000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_pred_round10_seed{seed}.csv", 
                                                         delimiter=",")
            
        except:
            pass
    
    # Transform back into vanilla dict
    d = dict(d)
    d["post_samples"] = dict(d["post_samples"])
    d["lik_samples"] = dict(d["lik_samples"])
    
    return d

In [ ]:
def load_samples_no_rounds(method_abbreviation, dir_prefix=""):
    d = defaultdict(dict)
    d['post_samples'] = defaultdict(dict)
    d['lik_samples'] = defaultdict(dict)

    for seed in config.seeds:
        d['post_samples']['2000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_sims2000_seed{seed}.csv", 
                                                     delimiter=",")
        d['post_samples']['6000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_sims6000_seed{seed}.csv", 
                                                     delimiter=",")
        d['post_samples']['10000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_sims10000_seed{seed}.csv", 
                                                     delimiter=",")
        try:
            d['lik_samples']['2000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_pred_sims2000_seed{seed}.csv", 
                                                         delimiter=",")
            d['lik_samples']['6000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_pred_sims6000_seed{seed}.csv", 
                                                         delimiter=",")
            d['lik_samples']['10000'][seed] = np.loadtxt(f"output/{dir_prefix}{method_abbreviation}_post_pred_sims10000_seed{seed}.csv", 
                                                         delimiter=",")
            
        except:
            pass
    
    # Transform back into vanilla dict
    d = dict(d)
    d["post_samples"] = dict(d["post_samples"])
    d["lik_samples"] = dict(d["lik_samples"])
    
    return d

In [ ]:
def load_samples_pkl_dict(method_abbreviation, dir_prefix=""):

    d = defaultdict(dict)
    d["post_samples"] = defaultdict(dict)
    d["lik_samples"] = defaultdict(dict)


    for n_simulations in ["2000", "6000", "10000"]:
        for seed in config.seeds:
            filename = (f"output/{dir_prefix}{method_abbreviation}_sims{n_simulations}_seed{seed}.pkl")
            with open(filename, "rb") as f:
                file_content = pickle.load(f)
                d["post_samples"][n_simulations][seed] = file_content["post_samples"]
                d["lik_samples"][n_simulations][seed] = file_content["lik_samples"]
            
    d = dict(d)
    d["post_samples"] = dict(d["post_samples"])
    d["lik_samples"] = dict(d["lik_samples"])
    
    return d

In [ ]:
def load_samples_dat(method_abbreviation, dir_prefix=""):

    d = defaultdict(dict)
    d["post_samples"] = defaultdict(dict)
    d["lik_samples"] = defaultdict(dict)


    for n_simulations in ["2000", "6000", "10000"]:
        for seed in config.seeds:
            filename = (f"output/{dir_prefix}{method_abbreviation}_attempt{seed}_{n_simulations}.dat")
            post_samples = pd.read_csv(filename, delimiter="\t", header=None).transpose().to_numpy()
            d["post_samples"][n_simulations][seed] = post_samples
            
    d = dict(d)
    d["post_samples"] = dict(d["post_samples"])
    d["lik_samples"] = dict(d["lik_samples"])
    
    return d

In [ ]:
def remove_out_of_prior(d):
    for n_simulations in d["post_samples"].keys():
        for i in range(1, len(d["post_samples"][n_simulations]) + 1):
            within_prior_idx = np.where((abs(d["post_samples"][n_simulations][i][:, 0]) < 2) & (abs(d["post_samples"][n_simulations][i][:, 1]) < 2))
            d["post_samples"][n_simulations][i] = d["post_samples"][n_simulations][i][within_prior_idx]
            d["post_samples"][n_simulations][i] = d["post_samples"][n_simulations][i][:1000, :]

            d["lik_samples"][n_simulations][i] = d["lik_samples"][n_simulations][i][within_prior_idx]
            d["lik_samples"][n_simulations][i] = d["lik_samples"][n_simulations][i][:1000, :]
    return d

# Posterior Scatterplot

In [ ]:
# plotting

import matplotlib.gridspec as gridspec
label_fontsize=22
scatter_kws = {'alpha': 0.5,
               'rasterized': True,
               's': 0.7,
               'color': (1, 1, 1, 0.64),
               'marker': 'D'}

def plot_twomoons_posterior(approximators, seed, analytic_posterior_samples, save_plot=False, dir_prefix=""):
    ncols = len(approximators) + 1
    nrows = 3
    
    fig, axes = plt.subplots(nrows=3, ncols=ncols, figsize=(2*ncols, 2*nrows), sharex=True, sharey=True)
    fig.delaxes(axes[1][0])
    fig.delaxes(axes[2][0])
    
    rows = [0, 1, 2]
    cols = list(range(1, ncols+1))
    n_simulations = ['2000', '6000', '10000']

    axes[0][0].scatter(analytic_posterior_samples[:,0], analytic_posterior_samples[:,1], **scatter_kws)
    axes[0][0].set_title("True",fontsize=label_fontsize)

    for (col, approximator) in zip(cols, approximators):
        axes[0][col].set_title(approximator.name, fontsize=label_fontsize)
    
        for (row, n) in zip (rows, n_simulations):
            x = approximator.samples['post_samples'][n][seed][:, 0]
            y = approximator.samples['post_samples'][n][seed][:, 1]
            axes[row, col].scatter(x, y, **scatter_kws)
    
    for (row, n) in zip (rows, n_simulations):
        axes[row][-1].set_ylabel(fr'$N = {n}$', fontsize=label_fontsize)
        axes[row][-1].yaxis.set_label_position("right")
    
    for ax in axes.flatten():
        ax.grid(False)
        ax.set_facecolor((0/255, 32/255, 64/255, 1.0))
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        ax.set_xlim(-2, 2)
        ax.set_ylim(-2, 2)
        ax.spines['bottom'].set_alpha(0.0)
        ax.spines['top'].set_alpha(0.0)
        ax.spines['right'].set_alpha(0.0)
        ax.spines['left'].set_alpha(0.0)
    
    if save_plot:
        plt.savefig(f"plots/{dir_prefix}tm_posterior_seed" + str(seed) + ".pdf", bbox_inches="tight")

# Posterior MMD Boxplot

In [ ]:
def plot_posterior_boxplot(approximators, n_simulations, analytic_posterior_samples, dir_prefix=""):
    n_seeds = len(config.seeds)
    approximators_filtered = list(filter(lambda a: a.has_posterior_samples, approximators))

    for approximator in approximators_filtered:
        approximator.posterior_mmd = np.zeros((n_seeds))
        for i, seed in enumerate(config.seeds):
            approximator.posterior_mmd[i] = float(maximum_mean_discrepancy(analytic_posterior_samples, 
                                                                                  approximator.samples["post_samples"][n_simulations][seed], 
                                                                                  kernel="gaussian", squared=False))
        
        print(f"{approximator.name} Mean (SD) = {np.mean(approximator.posterior_mmd).round(3)} ({np.std(approximator.posterior_mmd).round(3)})")

        
    post_mmd_df = pd.DataFrame({approximator.name: approximator.posterior_mmd for approximator in approximators_filtered})    

    
    fig, ax = plt.subplots(figsize=(10,4))
    ax.set_yticks(np.arange(0, 2, 0.5), minor=False)
    ax.set_yticks(np.arange(0, 2, 0.1), minor=True)
    ax.grid(axis='y', which="major", alpha=0.6)
    ax.grid(axis="y", which="minor", alpha=0.15)

    sns.boxplot(data=post_mmd_df, 
                ax=ax, width=0.6,
                flierprops={"marker": "o"},
                boxprops={"facecolor": (0.0, 0, 0.53, 0.3)},
                linewidth=2
               )

    ax.tick_params(axis="both", which="both", labelsize=24)
    sns.despine()
    plt.ylim(0, 1.5)
    plt.savefig(f"plots/{dir_prefix}tm_boxplot_posterior_nsim{n_simulations}.pdf", bbox_inches='tight')
    

# Posterior Predictive MMD Boxplot

In [ ]:
def plot_posterior_predictive_boxplot(approximators, n_simulations, analytic_posterior_predictive_samples, dir_prefix=""):
    n_seeds = len(config.seeds)
    approximators_filtered = list(filter(lambda a: a.has_posterior_predictive_samples, approximators))
    
    for approximator in approximators_filtered:
        approximator.posterior_predictive_mmd = np.zeros((n_seeds))
        for i, seed in enumerate(config.seeds):
            approximator.posterior_predictive_mmd[i] = float(maximum_mean_discrepancy(analytic_posterior_predictive_samples, 
                                                                                  approximator.samples["lik_samples"][n_simulations][seed], 
                                                                                  kernel="gaussian", squared=False))
        
        print(f"{approximator.name} Mean (SD) = {np.mean(approximator.posterior_predictive_mmd).round(3)} ({np.std(approximator.posterior_predictive_mmd).round(3)})")

        
    post_pred_mmd_df = pd.DataFrame({approximator.name: approximator.posterior_predictive_mmd for approximator in approximators_filtered})    
    
    fig, ax = plt.subplots(figsize=(4, 4))

    ax.set_yticks(np.arange(0, 0.21, 0.05), minor=False)
    ax.set_yticks(np.arange(0, 0.21, 0.01), minor=True)
    ax.grid(axis='y', which="major", alpha=0.6)
    ax.grid(axis="y", which="minor", alpha=0.15)
    ax.set_ylim(0.00, 0.2)

    sns.boxplot(data=post_pred_mmd_df, 
                ax=ax, width=0.6,
                flierprops={"marker": "o"},
                boxprops={"facecolor": (0.53, 0, 0, 0.30)},
                linewidth=2
               )

    ax.tick_params(axis="both", which="both", labelsize=24)
    sns.despine()

    plt.savefig(f"plots/{dir_prefix}tm_boxplot_posterior_predictive_nsim{n_simulations}.pdf", bbox_inches='tight')  

# Full Evaluation Function, parameterized by the simulator

In [ ]:
class Approximator:
    def __init__(self, name, abbreviation, sample_loader, has_posterior_samples=True, has_posterior_predictive_samples=False):
        self.name = name
        self.abbreviation = abbreviation
        self.sample_loader = sample_loader
        self.has_posterior_samples = has_posterior_samples
        self.has_posterior_predictive_samples = has_posterior_predictive_samples
        self.samples = None
        self.posterior_mmd = None
        self.posterior_predictive_mmd = None

    def load_samples(self, dir_prefix=""):
        self.samples = self.sample_loader(self.abbreviation, dir_prefix)

In [ ]:
def two_moons_evaluate_and_plot(approximators,
                                analytic_posterior_sampler, 
                                analytic_likelihood_sampler,
                                dir_prefix=""):
    # Load samples from approximators
    for approximator in approximators:
        approximator.load_samples(dir_prefix)
    
    # Generate analytic samples as ground truth
    x_o = np.array([0, 0])
    analytic_posterior_samples = analytic_posterior_sampler(x_o, 1000)
    analytic_posterior_predictive_samples = analytic_likelihood_sampler(analytic_posterior_samples)
    
    # Posterior: Scatterplot
    for seed in config.seeds:
        plot_twomoons_posterior(approximators=approximators, 
                                seed=seed, 
                                analytic_posterior_samples=analytic_posterior_samples,
                                save_plot=True, 
                                dir_prefix=dir_prefix)
    
    # Posterior MMD and Posterior Predictive MMD
    for n_simulations in ['2000', '6000', '10000']:
        plot_posterior_boxplot(approximators=approximators,
                               n_simulations=n_simulations, 
                               analytic_posterior_samples=analytic_posterior_samples,
                               dir_prefix=dir_prefix)
        
        plot_posterior_predictive_boxplot(approximators=approximators,
                                          n_simulations=n_simulations, 
                                          analytic_posterior_predictive_samples=analytic_posterior_predictive_samples,
                                          dir_prefix=dir_prefix)

    return analytic_posterior_samples, analytic_posterior_predictive_samples

In [ ]:
snpe_c = Approximator(name="SNPE-C", abbreviation="snpec", sample_loader=load_samples_np_txt, 
                   has_posterior_samples=True, has_posterior_predictive_samples=False)
snre_b = Approximator(name="SNRE-B", abbreviation="snreb", sample_loader=load_samples_np_txt, 
                   has_posterior_samples=True, has_posterior_predictive_samples=False)
snpla = Approximator(name="SNPLA", abbreviation="snpla", sample_loader=load_samples_np_txt, 
                   has_posterior_samples=True, has_posterior_predictive_samples=True)
snvi = Approximator(name="SNVI", abbreviation="snvi", sample_loader=load_samples_np_txt, 
                   has_posterior_samples=True, has_posterior_predictive_samples=True)
npe_c = Approximator(name="NPE-C", abbreviation="npec", sample_loader=load_samples_no_rounds,
                    has_posterior_samples=True, has_posterior_predictive_samples=False)
gsmc = Approximator(name="gSMC", abbreviation="postABCdraws_hybrid", sample_loader=load_samples_dat, 
                   has_posterior_samples=True, has_posterior_predictive_samples=False)
jana = Approximator(name="JANA", abbreviation="jana", sample_loader=load_samples_no_rounds, 
                   has_posterior_samples=True, has_posterior_predictive_samples=True)

# Simulator 1: Wiqvist et al

In [ ]:
approximators = [gsmc, npe_c, snre_b, snpe_c, snvi, snpla, jana]

post_samples_1, post_pred_samples_1 = two_moons_evaluate_and_plot(approximators=approximators,
                            analytic_posterior_sampler=two_moons_wiqvist_numpy.analytic_posterior_numpy,
                            analytic_likelihood_sampler=two_moons_wiqvist_numpy.simulator_numpy_batched,
                            dir_prefix="simulator_1/")

# Simulator 2: Lueckmann et al

In [ ]:
approximators = [npe_c, snre_b, snpe_c, snvi, snpla, jana]
two_moons_evaluate_and_plot(approximators=approximators,
                            analytic_posterior_sampler=two_moons_lueckmann_numpy.analytic_posterior_numpy,
                            analytic_likelihood_sampler=two_moons_lueckmann_numpy.simulator_numpy_batched,
                            dir_prefix="simulator_2/")

# Custom Plots for the Paper

In [ ]:
analytic_posterior_samples = post_samples_1
analytic_posterior_predictive_samples = post_pred_samples_1

In [ ]:
# load simulator_1 results again, treat SNL separately because only one repetition
seed = 1
dir_prefix = "simulator_1/"

snl = Approximator(name="SNL", abbreviation="snl", sample_loader=load_samples_np_txt, 
                   has_posterior_samples=True, has_posterior_predictive_samples=True)

snl.samples = load_samples_np_txt(method_abbreviation="snl", dir_prefix="simulator_1/", seeds=[1])

for approximator in [gsmc, npe_c, snre_b, snpe_c, snvi, snpla, jana]:
    approximator.load_samples(dir_prefix)

approximators = [gsmc, npe_c, snl, snre_b, snpe_c, snvi, snpla, jana]

In [ ]:
import matplotlib.gridspec as gridspec
label_fontsize=22
scatter_kws = {'alpha': 0.5,
               'rasterized': True,
               's': 0.7,
               'color': (1, 1, 1, 0.64),
               'marker': 'D'}



ncols = len(approximators) + 1
nrows = 3

fig, axes = plt.subplots(nrows=3, ncols=ncols, figsize=(2*ncols, 2*nrows), sharex=True, sharey=True)
fig.delaxes(axes[1][0])
fig.delaxes(axes[2][0])

rows = [0, 1, 2]
cols = list(range(1, ncols+1))
n_simulations = ['2000', '6000', '10000']

axes[0][0].scatter(analytic_posterior_samples[:,0], analytic_posterior_samples[:,1], **scatter_kws)
axes[0][0].set_title("True",fontsize=label_fontsize)

for (col, approximator) in zip(cols, approximators):
    axes[0][col].set_title(approximator.name, fontsize=label_fontsize)

    for (row, n) in zip (rows, n_simulations):
        x = approximator.samples['post_samples'][n][seed][:, 0]
        y = approximator.samples['post_samples'][n][seed][:, 1]
        axes[row, col].scatter(x, y, **scatter_kws)

for (row, n) in zip (rows, n_simulations):
    axes[row][-1].set_ylabel(fr'$N = {n}$', fontsize=label_fontsize)
    axes[row][-1].yaxis.set_label_position("right")

for ax in axes.flatten():
    ax.grid(False)
    ax.set_facecolor((0/255, 32/255, 64/255, 1.0))
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.spines['bottom'].set_alpha(0.0)
    ax.spines['top'].set_alpha(0.0)
    ax.spines['right'].set_alpha(0.0)
    ax.spines['left'].set_alpha(0.0)

plt.savefig(f"plots/{dir_prefix}tm_posterior_seed" + str(seed) + ".pdf", bbox_inches="tight")

In [ ]:
seeds = list(range(1, 11))
n_seeds = len(seeds)
approximators = [gsmc, npe_c, snre_b, snpe_c, snvi, snpla, jana]
approximators_filtered = list(filter(lambda a: a.has_posterior_samples, approximators))

for n_simulations in ['2000', '6000', '10000']:

    for approximator in approximators_filtered:
        approximator.posterior_mmd = np.zeros((n_seeds))
        for i, seed in enumerate(seeds):
            approximator.posterior_mmd[i] = float(maximum_mean_discrepancy(analytic_posterior_samples, 
                                                                                  approximator.samples["post_samples"][n_simulations][seed], 
                                                                                  kernel="gaussian", squared=False))

        print(f"{approximator.name} Mean (SD) = {np.mean(approximator.posterior_mmd).round(3)} ({np.std(approximator.posterior_mmd).round(3)})")


    snl_mmd = float(maximum_mean_discrepancy(analytic_posterior_samples,
                                            snl.samples['post_samples'][n_simulations][1],
                                            kernel="gaussian", squared=False))

    post_mmd_df = pd.DataFrame({approximator.name: approximator.posterior_mmd for approximator in approximators_filtered})    
    post_mmd_df.insert(4, 'SNL', [snl_mmd] * n_seeds)

    fig, ax = plt.subplots(figsize=(6,4))
    ax.set_yticks(np.arange(0, 2, 0.5), minor=False)
    ax.set_yticks(np.arange(0, 2, 0.1), minor=True)
    ax.grid(axis='y', which="major", alpha=0.6)
    ax.grid(axis="y", which="minor", alpha=0.15)

    sns.boxplot(data=post_mmd_df, 
                ax=ax, width=0.6,
                flierprops={"marker": "o"},
                boxprops={"facecolor": (0.0, 0, 0.53, 0.3)},
                linewidth=1.5,

               )

    ax.tick_params(axis="both", which="both", labelsize=24)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    sns.despine()
    plt.ylim(0, 1.3)
    plt.savefig(f"plots/{dir_prefix}tm_boxplot_posterior_nsim{n_simulations}.pdf", bbox_inches='tight')


In [ ]:
seeds = list(range(1, 11))
n_seeds = len(seeds)
approximators = [gsmc, npe_c, snre_b, snpe_c, snvi, snpla, jana]

for n_simulations in ['2000', '6000', '10000']:
    approximators_filtered = list(filter(lambda a: a.has_posterior_predictive_samples, approximators))
    
    for approximator in approximators_filtered:
        approximator.posterior_predictive_mmd = np.zeros((n_seeds))
        for i, seed in enumerate(config.seeds):
            approximator.posterior_predictive_mmd[i] = float(maximum_mean_discrepancy(analytic_posterior_predictive_samples, 
                                                                                  approximator.samples["lik_samples"][n_simulations][seed], 
                                                                                  kernel="gaussian", squared=False))
        
        print(f"{approximator.name} Mean (SD) = {np.mean(approximator.posterior_predictive_mmd).round(3)} ({np.std(approximator.posterior_predictive_mmd).round(3)})")

        
    post_pred_mmd_df = pd.DataFrame({approximator.name: approximator.posterior_predictive_mmd for approximator in approximators_filtered})    

    snl_mmd = float(maximum_mean_discrepancy(analytic_posterior_predictive_samples,
                                            snl.samples['lik_samples'][n_simulations][1],
                                            kernel="gaussian", squared=False))

    post_pred_mmd_df.insert(0, 'SNL', [snl_mmd] * n_seeds)
    
    
    fig, ax = plt.subplots(figsize=(3, 4))

    ax.set_yticks(np.arange(0, 0.25, 0.05), minor=False)
    ax.set_yticks(np.arange(0, 0.25, 0.01), minor=True)
    ax.grid(axis='y', which="major", alpha=0.6)
    ax.grid(axis="y", which="minor", alpha=0.15)
    ax.set_ylim(0.10, 0.17)

    sns.boxplot(data=post_pred_mmd_df, 
                ax=ax, width=0.6,
                flierprops={"marker": "o"},
                boxprops={"facecolor": (0.53, 0, 0, 0.30)},
                linewidth=1.5
               )

    ax.tick_params(axis="both", which="both", labelsize=24)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    sns.despine()

    plt.savefig(f"plots/{dir_prefix}tm_boxplot_posterior_predictive_nsim{n_simulations}.pdf", bbox_inches='tight') 

In [ ]:
for amortizer in [npe_c, snre_b, snpe_c, snvi, snpla, jana]:
    dfs = [] 
    for seed in config.seeds:
        filename = f"output/simulator_1/{amortizer.abbreviation}_metadata_seed{seed}.txt"
        df = pd.read_csv(filename, delimiter=":", header=None).transpose()
        df.columns = df.iloc[0]
        df = df[1:]
        df.reset_index(drop=True, inplace=True)
        dfs.append(df)
    wall_clock_times = pd.concat(dfs)
    means = wall_clock_times.mean(axis=0)
    try:
        print(f"{amortizer.name} training: {means[0]:.0f}, post: {means[1]:.2f}, postpred: {means[2]:.2f}")
    except:
        print(f"{amortizer.name} training: {means[0]:.0f}, post: {means[1]:.2f}")